In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType
from delta.tables import DeltaTable

TABELA_ORIGEM = "v_credit.bronze.chamados"
TABELA_CHAMADO_LOG = "v_credit.silver.tb_chamado_log"
TABELA_MOTIVO = "v_credit.silver.tb_motivo"
TABELA_CANAL = "v_credit.silver.tb_canal"

TABELA_DESTINO_VALIDOS = "v_credit.silver.tb_chamado"
TABELA_DESTINO_INVALIDOS = "v_credit.silver.tb_chamado_invalidos"

CORRECOES_MOTIVO = {
    r"Bloqueio de cart.*o": "Bloqueio de cartão",
    r"Contesta..o de fatura": "Contestação de fatura",
    r"D.vidas gerais sobre programa de pontos": "Dúvidas gerais sobre programa de pontos",
    r"Compra n.o autorizada": "Compra não autorizada", 
    r"Contrata..o de cart.o adicional": "Contratação de cartão adicional",
    r"Altera..o de dados cadastrais.*": "Alteração de dados cadastrais (vencimento da fatura, telefone, email)",
    r"Desbloqueio de cart.o": "Desbloqueio de cartão",
    r"Transfer.ncia de ag.ncia": "Transferência de agência",
    r"Renegocia..o de d.vida": "Renegociação de dívida",
    r"Problema com aplicativo": "Problema com aplicativo"
}

CORRECOES_CANAL = {
    r"(?i)U\.?R\.?A\.?": "URA",
    r"(?i)CHAT\s?BOT": "CHATBOT",
    r"(?i)^ATEND.*INICIAL$": "ATENDIMENTO INICIAL",
    r"(?i)^ATEND.*ESPECIALIZADO$": "ATENDIMENTO ESPECIALIZADO"
}

def buscar_ultimo_dado(table_name):
    """Lê a tabela e filtra apenas os registros com o maior ingestion_timestamp."""
    df = spark.table(table_name)
    
    if "ingestion_timestamp" in df.columns:
        max_ts = (
            df
            .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
            .first()["max_ts"]
        )

        df_latest = df.filter(F.col("ingestion_timestamp") == F.lit(max_ts))
        print(f"✅ Tabela {table_name}: Filtrada pela carga mais recente: {max_ts}")
        return df_latest
    else:
        print(f"⚠️ Tabela {table_name}: Coluna 'ingestion_timestamp' não encontrada. Retornando dados completos.")
        return df

df_bronze = buscar_ultimo_dado(TABELA_ORIGEM)
df_chamado_log = buscar_ultimo_dado(TABELA_CHAMADO_LOG)
df_motivo = buscar_ultimo_dado(TABELA_MOTIVO)
df_canal = buscar_ultimo_dado(TABELA_CANAL)

### Configuracao
Define origens, destinos e dicionarios de correcao para motivos e canais (tratamento de typos e padronizacao).

### Leitura Incremental de Todas as Tabelas
Le ultima carga de:
- Bronze chamados (tabela principal)
- Silver tb_chamado_log (timestamps)
- Silver tb_motivo (lookup para cd_motivo)
- Silver tb_canal (lookup para cd_canal)

### Correcao de Typos
Aplica dicionarios de regex para corrigir:
- **Motivos**: Typos de acentuacao (ex: "cart.o" → "cartão")
- **Canais**: Padronizacao (ex: "U.R.A." → "URA", "chat bot" → "CHATBOT")

### Joins com Tabelas de Lookup
Faz LEFT JOIN para enriquecer chamados com:
- tb_chamado_log: Timestamps (dh_abertura, dh_inicio, dh_fim)
- tb_motivo: Codigo do motivo (cd_motivo)
- tb_canal: Codigo do canal (cd_canal)

**Importante**: LEFT JOIN permite rastrear chamados que nao tem match (vao para tabela de invalidos).

### Transformacao e Calculos
Aplica transformacoes complexas:
- **cd_cliente**: Hash SHA256 (anonimizacao LGPD)
- **st_resolvido**: Mapeia "SIM"/"NAO" para Boolean
- **tm_espera**: Calcula diferenca (dh_inicio - dh_abertura) em segundos
- **tm_duracao**: Calcula diferenca (dh_fim - dh_inicio) em segundos
- **cd_atendente**: NULL se vazio (indica atendimento por Bot)

### Validacao de Integridade Referencial
Adiciona flags de validacao para Foreign Keys:
- **flag_pk_valida**: cd_chamado IS NOT NULL
- **flag_fk_cliente_valida**: cd_cliente IS NOT NULL
- **flag_fk_motivo_valida**: cd_motivo foi encontrado no join com tb_motivo
- **flag_fk_canal_valida**: cd_canal foi encontrado no join com tb_canal

Split em df_validos e df_invalidos.

### MERGE e Auditoria
Faz MERGE dos registros validos em tb_chamado e OVERWRITE dos invalidos em tb_chamado_invalidos.

# Transformacao Bronze → Silver: tb_chamado

## Proposito
Este notebook transforma dados brutos de chamados (tickets) da camada Bronze para Silver, aplicando **limpeza, validacao e split de qualidade**.

## Por que tb_chamado e critico?
Esta e a **tabela fato principal** do projeto, contendo:
- Todas as transacoes de atendimento
- Metricas de tempo (espera, duracao)
- Relacionamentos com clientes, canais e atendentes

## O que este notebook faz:

### 1. Extracao Incremental
- Le apenas registros com `ingestion_timestamp` mais recente
- Evita reprocessamento desnecessario

### 2. Limpeza de Dados
- Renomeia colunas para padrao Silver (`cd_`, `ds_`, `tm_`)
- Remove espacos e normaliza textos
- Converte tipos de dados
- Remove duplicatas por Primary Key

### 3. Validacao de Qualidade
Aplica regras de negocio:
- **flag_id_valido**: cd_chamado NOT NULL
- **flag_cliente_valido**: cd_cliente NOT NULL
- **flag_canal_valido**: cd_canal NOT NULL
- **flag_tempos_validos**: tempos >= 0

### 4. Split de Dados
- **Registros VALIDOS** (`flag_qualidade = "OK"`):
  - Vao para `tb_chamado` via **MERGE** (upsert)
- **Registros INVALIDOS** (`flag_qualidade = "ERRO"`):
  - Vao para `tb_chamado_invalidos` via **OVERWRITE** para auditoria

## Decisoes Tecnicas:

### Por que MERGE ao inves de Overwrite?
- **Idempotencia**: Posso re-executar sem duplicar dados
- **Incremental**: Suporta cargas diarias sem reprocessar tudo

### Por que tabelas de auditoria?
- **Rastreabilidade**: Saber quais registros falharam
- **Analise de Causa**: Identificar padroes de erro
- **Correcao na Origem**: Feedback para corrigir no Supabase
- **Nao Bloqueio**: Pipeline continua mesmo com erros

## Metricas de Negocio:
- **CSAT**: Satisfacao do cliente
- **Tempo de Espera**: Critico para experiencia
- **Tempo de Atendimento**: Eficiencia operacional
- **Taxa de Resolucao**: % de chamados resolvidos

## Dependencias:
- Bronze carregado
- DDL Silver executado (`ddl_tb_chamado.ipynb` e `ddl_tb_chamado_invalidos.ipynb`)

## Proximo passo:
Carregar camada Gold para modelagem dimensional.

In [0]:
df_motivo_corrigido = df_bronze
for pattern, replacement in CORRECOES_MOTIVO.items():
    df_motivo_corrigido = df_motivo_corrigido.withColumn(
        "motivo", F.regexp_replace(F.col("motivo"), pattern, replacement)
    )

df_canal_corrigido = df_motivo_corrigido
for pattern, replacement in CORRECOES_CANAL.items():
    df_canal_corrigido = df_canal_corrigido.withColumn(
        "canal", F.regexp_replace(F.upper(F.trim(F.col("canal"))), pattern, replacement)
    )


In [0]:
df_com_lookups = (
    df_canal_corrigido.alias("c")
    .join(
        df_chamado_log.select("cd_chamado", "dh_abertura", "dh_inicio", "dh_fim").alias("log"),
        F.col("c.id_chamado") == F.col("log.cd_chamado"),
        "left"
    )
    .join(
        df_motivo.select("cd_motivo", "ds_motivo").alias("m"),
        F.lower(F.trim(F.col("c.motivo"))) == F.lower(F.trim(F.col("m.ds_motivo"))),
        "left"
    )
    .join(
        df_canal.select("cd_canal", "nm_canal").alias("can"),
        F.trim(F.col("c.canal")) == F.col("can.nm_canal"),
        "left"
    )
)

In [0]:
df_transformado = (
    df_com_lookups
    .select(
        F.col("c.id_chamado").cast(LongType()).alias("cd_chamado"),

        F.sha2(F.col("c.id_cliente").cast(StringType()), 256).alias("cd_cliente"),

        F.col("m.cd_motivo").cast(LongType()).alias("cd_motivo"),
        F.col("can.cd_canal").cast(LongType()).alias("cd_canal"),

        F.when(
            F.upper(F.regexp_replace(F.col("c.resolvido"), r"(?i)n.o", "NAO")) == "SIM",
            F.lit(True)
        ).when(
            F.upper(F.regexp_replace(F.col("c.resolvido"), r"(?i)n.o", "NAO")) == "NAO",
            F.lit(False)
        ).otherwise(None).cast(BooleanType()).alias("st_resolvido"),

        (F.col("log.dh_inicio").cast("long") - F.col("log.dh_abertura").cast("long")).alias("tm_espera"),
        (F.col("log.dh_fim").cast("long") - F.col("log.dh_inicio").cast("long")).alias("tm_duracao"),

        F.when(
            (F.col("c.id_atendente").isNull()) |
            (F.trim(F.col("c.id_atendente").cast(StringType())) == ""),
            F.lit(None)
        ).otherwise(F.col("c.id_atendente").cast(LongType())).alias("cd_atendente"),

        F.col("c.ingestion_timestamp").alias("dt_ingestion"),
        F.coalesce(F.col("c.origem"), F.lit("chamados")).alias("dc_origem")
    )
    .dropDuplicates(["cd_chamado"])
)


In [0]:
df_validacao = (
    df_transformado
    .withColumn("flag_pk_valida", F.col("cd_chamado").isNotNull())
    .withColumn("flag_fk_cliente_valida", F.col("cd_cliente").isNotNull())
    .withColumn("flag_fk_motivo_valida", F.col("cd_motivo").isNotNull())
    .withColumn("flag_fk_canal_valida", F.col("cd_canal").isNotNull())
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_pk_valida") &
            F.col("flag_fk_cliente_valida") &
            F.col("flag_fk_motivo_valida") &
            F.col("flag_fk_canal_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_silver = (
    df_validacao
    .filter(F.col("flag_qualidade") == "OK")
    .select(
        "cd_chamado", "cd_cliente", "cd_motivo", "cd_canal", "st_resolvido",
        "tm_espera", "tm_duracao", "cd_atendente", "dt_ingestion", "dc_origem"
    )
)

df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

total_validos = df_silver.count()
total_invalidos = df_invalidos.count()

print(f"✅ Transformação e validação concluídas: {total_validos} válidos | {total_invalidos} inválidos")

In [0]:
delta_table = DeltaTable.forName(spark, TABELA_DESTINO_VALIDOS)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_chamado = origem.cd_chamado"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {TABELA_DESTINO_VALIDOS} atualizada com sucesso!")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(TABELA_DESTINO_INVALIDOS)
print(f"✅ Tabela {TABELA_DESTINO_INVALIDOS} atualizada para auditoria!")